In [1]:
# Importing Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Setting the Path
path = 'COVID'

In [3]:
# Loading the images
train_data_dir = os.path.join(path, 'Train')
test_data_dir = os.path.join(path, 'Test')

In [4]:
# Image Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


In [5]:
# Training the Model
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150,150),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(150,150),
    batch_size=16,
    class_mode='binary',
    shuffle=False)

Found 1800 images belonging to 2 classes.
Found 681 images belonging to 2 classes.


In [6]:
model = Sequential()

#Adding convolutional layers
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Adding a second convolutional layer
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [7]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
# Training the Model
model.fit(
    train_generator,
    steps_per_epoch=40,
    epochs=50,
    validation_data=test_generator,
    validation_steps=32)

Epoch 1/50
40/40 [==============================] - 49s 1s/step - loss: 0.7612 - accuracy: 0.5203 - val_loss: 0.6821 - val_accuracy: 0.6914
Epoch 2/50
40/40 [==============================] - 44s 1s/step - loss: 0.6803 - accuracy: 0.5969 - val_loss: 0.7232 - val_accuracy: 0.3613
Epoch 3/50
40/40 [==============================] - 42s 1s/step - loss: 0.5931 - accuracy: 0.7000 - val_loss: 0.5801 - val_accuracy: 0.6758
Epoch 4/50
40/40 [==============================] - 41s 1s/step - loss: 0.5391 - accuracy: 0.7609 - val_loss: 0.4932 - val_accuracy: 0.7637
Epoch 5/50
40/40 [==============================] - 47s 1s/step - loss: 0.4617 - accuracy: 0.7848 - val_loss: 0.5556 - val_accuracy: 0.6699
Epoch 6/50
40/40 [==============================] - 33s 830ms/step - loss: 0.4496 - accuracy: 0.7896 - val_loss: 0.5842 - val_accuracy: 0.6602
Epoch 7/50
40/40 [==============================] - 43s 1s/step - loss: 0.3767 - accuracy: 0.8547 - val_loss: 0.4670 - val_accuracy: 0.8027
Epoch 8/50
40/40 

In [9]:
model.evaluate(test_generator,steps=40)

40/40 [==============================] - 5s 130ms/step - loss: 1.5223 - accuracy: 0.7531


[1.522275686264038, 0.753125011920929]

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [11]:
# Saving the Model
model.save('COVID_detection_model.h5')

In [12]:
model = keras.models.load_model('COVID_detection_model.h5')

In [13]:
# Creating the GUI
from tkinter import *
from PIL import Image, ImageTk

import tkinter
import tkinter as tk
import customtkinter
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
from tkinter import messagebox
import keras.utils as image
import numpy as np
import os

customtkinter.set_appearance_mode("System")
root = customtkinter.CTk()

#window size
root.rowconfigure(0,weight=1)
root.columnconfigure(0,weight=1)

root.geometry('420x270')

root.title('COVID Detection')

def reset_btn():
    global label_prediction
    
    result = tk.Label(root, text="                                             ")
    result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)

# Function to Select Image
def browse_btn():
    global image_name
    image_name = askopenfilename(title='Select Image')
    
    img = Image.open(image_name)
    img = img.resize((200, 200), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    panel = tk.Label(root, image=img)
    panel.image = img
    panel.grid(row=0,column=1,sticky='nw',padx=20,pady=20)

# Function to Predict
def predict_btn():
    global label_prediction
    global image_name
    test_img = image.load_img(image_name, target_size=(150, 150))
    test_img = image.img_to_array(test_img)
    test_img = np.expand_dims(test_img, axis=0)
    result = model.predict(test_img)
    print(result)
    if result[0][0] == 0:
        result = tk.Label(root, text="COVID Detected")
        result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)
    elif result[0][0] == 1:
        result = tk.Label(root, text="No COVID Detected")
        result.grid(row=1,column=1,sticky='nw',padx=20,pady=20)

# Browse Button
browsebtn = customtkinter.CTkButton(master=root, text="Browse Image", command=browse_btn)
browsebtn.grid(row=0, column=0,sticky='nw',padx=20,pady=20)

# Predict Butoon
predictbtn = customtkinter.CTkButton(master=root, text="Predict", command=predict_btn)
predictbtn.grid(row=1, column=0,sticky='nw',padx=20,pady=20)
predictbtn = customtkinter.CTkButton(master=root, text="Reset", command=reset_btn)
predictbtn.grid(row=2, column=0,sticky='nw',padx=20,pady=20)




# Running the GUI
root.mainloop()

1/1 [==============================] - 0s 143ms/step
[[1.]]
1/1 [==============================] - 0s 30ms/step
[[1.]]
1/1 [==============================] - 0s 28ms/step
[[1.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 29ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
[[1.]]
1/1 [==============================] - 0s 28ms/step
[[1.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
1/1 [==============================] - 0s 28ms/step
[[0.]]
